In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
# import libraries ต่าง ๆ ที่จะใช้
import torch
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split # ใช้ในการแบ่งข้อมูล

In [4]:
df1 = pd.read_csv('check0.csv',index_col=0)
df2 = pd.read_csv('check1.csv',index_col=0)

In [2]:
# import dataset
df = pd.read_csv("../3 - Cleaning & Transforming Data/1 cleaned_dataset_MERGED_DRUG(Mogran)_ALLELESEQ.csv", index_col = None)
df.drop(columns=['Drug','Allele','No_ADR','ADR','ADR_MPeruption','ADR_MPexanthema','ADR_SJS','ADR_TEN','Cohort ethnicity'], inplace=True)

In [11]:
df.dtypes.value_counts()

int64      4089
float64       1
dtype: int64

In [3]:
# define get_dataset()
def get_dataset(df, BATCHSIZE, outputcol):
    #from torchsampler import ImbalancedDatasetSampler
    df2 = df.copy()
    df_labels = df2[outputcol]
    df2.drop(columns=outputcol, inplace=True)
    
    df_input = df.iloc[:, 1:]

    np_input = df_input.to_numpy()
    np_labels = df_labels.to_numpy()

    #train test split
    train_data, test_data, train_labels, test_labels = train_test_split(np_input, np_labels, test_size=0.2, random_state=128, stratify=np_labels, shuffle = True)
    
    print('train_label data distribution 0/1: {}/{}'.format(
    len(np.where(train_labels == 0)[0]), len(np.where(train_labels == 1)[0])))
    
    print('test_labels data distribution 0/1: {}/{}'.format(
    len(np.where(test_labels == 0)[0]), len(np.where(test_labels == 1)[0])))

    #calculate weight for each class in train_labels
    weight = (1/pd.DataFrame(train_labels).value_counts()).tolist()
    
    #create weight array for train_labels
    sample_weights = np.array([weight[int(t)] for t in train_labels])
    
    #assign weight to Weighted Random Sampler
    sampler = WeightedRandomSampler(weights=sample_weights,num_samples=len(sample_weights), replacement=True)
    
    #แปลง numpy เข้า TensorDataSet และ DataLoader
    train_dataset = TensorDataset(torch.from_numpy(train_data).float(), torch.from_numpy(train_labels).float())
    test_dataset = TensorDataset(torch.from_numpy(test_data).float(), torch.from_numpy(test_labels).float())
    train_loader = DataLoader(train_dataset, batch_size=BATCHSIZE, sampler=sampler)
    test_loader = DataLoader(test_dataset, batch_size=test_labels.shape[0])

    #loop ดูภาพรวมของ train_loader
    for i, (data, target) in enumerate(train_loader):
        print("batch index {}, 0/1: {}/{}".format(
            i,
            len(np.where(target.numpy() == 0)[0]),
            len(np.where(target.numpy() == 1)[0])))

    return train_loader, test_loader

In [4]:
train_loader, test_loader = get_dataset(df,32,'Yes_ADR')

train_label data distribution 0/1: 6750/3708
test_labels data distribution 0/1: 1688/927
batch index 0, 0/1: 15/17
batch index 1, 0/1: 13/19
batch index 2, 0/1: 21/11
batch index 3, 0/1: 22/10
batch index 4, 0/1: 16/16
batch index 5, 0/1: 16/16
batch index 6, 0/1: 17/15
batch index 7, 0/1: 19/13
batch index 8, 0/1: 17/15
batch index 9, 0/1: 14/18
batch index 10, 0/1: 20/12
batch index 11, 0/1: 19/13
batch index 12, 0/1: 17/15
batch index 13, 0/1: 16/16
batch index 14, 0/1: 15/17
batch index 15, 0/1: 21/11
batch index 16, 0/1: 15/17
batch index 17, 0/1: 16/16
batch index 18, 0/1: 20/12
batch index 19, 0/1: 16/16
batch index 20, 0/1: 16/16
batch index 21, 0/1: 16/16
batch index 22, 0/1: 15/17
batch index 23, 0/1: 19/13
batch index 24, 0/1: 19/13
batch index 25, 0/1: 15/17
batch index 26, 0/1: 20/12
batch index 27, 0/1: 17/15
batch index 28, 0/1: 18/14
batch index 29, 0/1: 16/16
batch index 30, 0/1: 14/18
batch index 31, 0/1: 16/16
batch index 32, 0/1: 19/13
batch index 33, 0/1: 13/19
bat

In [9]:
test_model = nn.Sequential(
    nn.Linear(train_loader.dataset.tensors[0].shape[1], 637),
    nn.ReLU(),
    nn.Linear(637, 3513),
    nn.ReLU(),
    nn.Linear(3513,1015),
    nn.ReLU(),
    nn.Linear(1015,3136),
    nn.ReLU(),
    nn.Linear(3136, 328),
    nn.ReLU(),
    nn.Linear(328, 1),
    nn.Sigmoid()
)
test_model = test_model

test_model.load_state_dict(torch.load('../4 - Training & Testing/logs_test_final_3_MORGAN/trial-0_model.pt'))

correct = 0
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        # # Limiting validation data.
        # if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
        #     break
        data = data.float()
        target = target.float()             
        output = test_model(data)
        output = output.flatten()
        #data = data.flatten()
        #test_loss = loss_fn(output, target)
        output = torch.round(output)
        correct = (output == target).sum().item()
        
        # # Get the index of the max log-probability.
        # pred = output.argmax(dim=1, keepdim=True)
        # correct += pred.eq(target.view_as(pred)).sum().item()
#accuracy = correct/len(valid_loader.dataset)
    test_acc = correct / len(test_loader.dataset) #min(len(test_loader.dataset),N_VALID_EXAMPLES)

test_acc


1.0